In [1]:
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
from implementations import * 

In [2]:
data_path = "data/train.csv"

In [3]:
y, x, ids = load_csv_data(data_path, sub_sample=False)

In [4]:
tx, mean_x, std_x = standardize(x)

In [5]:
#SGD parameters
initial_w = np.zeros((x.shape[1]))
max_iters = 100
gamma = 0.01
lambda_ = 0.7

In [10]:
w_ls_sg, loss_ls_gd = least_squares_GD(y, tx, initial_w, max_iters, gamma)

Gradient Descent(0/99): loss=0.5000000000
Gradient Descent(1/99): loss=0.4401420042
Gradient Descent(2/99): loss=0.4367786984
Gradient Descent(3/99): loss=0.4349683956
Gradient Descent(4/99): loss=0.4333423494
Gradient Descent(5/99): loss=0.4318501655
Gradient Descent(6/99): loss=0.4304762522
Gradient Descent(7/99): loss=0.4292078895
Gradient Descent(8/99): loss=0.4280341066
Gradient Descent(9/99): loss=0.4269453800
Gradient Descent(10/99): loss=0.4259334015
Gradient Descent(11/99): loss=0.4249908885
Gradient Descent(12/99): loss=0.4241114286
Gradient Descent(13/99): loss=0.4232893508
Gradient Descent(14/99): loss=0.4225196188
Gradient Descent(15/99): loss=0.4217977432
Gradient Descent(16/99): loss=0.4211197069
Gradient Descent(17/99): loss=0.4204819036
Gradient Descent(18/99): loss=0.4198810859
Gradient Descent(19/99): loss=0.4193143206
Gradient Descent(20/99): loss=0.4187789520
Gradient Descent(21/99): loss=0.4182725695
Gradient Descent(22/99): loss=0.4177929803
Gradient Descent(23/9

In [23]:
w_ls_sg

array([ 0.2035595 , -0.12962203, -0.05527349,  0.01229099, -0.02450509,
        0.19385846, -0.02780161, -0.02976163, -0.0456469 , -0.03147113,
       -0.03162885, -0.02774737, -0.02520986,  0.00993751, -0.02963122,
       -0.02967984, -0.05291429, -0.02962913, -0.02952568, -0.0311228 ,
       -0.02949501, -0.05152506, -0.03020389,  0.02824165,  0.01292121,
        0.01291545, -0.03789906, -0.02545391, -0.02550089, -0.04770133])

In [11]:
w_ls_sgd, loss_ls_sgd = least_squares_SGD(y, tx, initial_w, max_iters, gamma)

Gradient Descent(0/99): loss=1.0000000000
Gradient Descent(1/99): loss=1.3057120178
Gradient Descent(2/99): loss=1.0654846161
Gradient Descent(3/99): loss=2.4556311668
Gradient Descent(4/99): loss=3.5352898015
Gradient Descent(5/99): loss=2.6580898394
Gradient Descent(6/99): loss=1.4466263320
Gradient Descent(7/99): loss=1.9062438208
Gradient Descent(8/99): loss=1.6942988594
Gradient Descent(9/99): loss=1.6944467485
Gradient Descent(10/99): loss=2.8010595636
Gradient Descent(11/99): loss=1.3413237539
Gradient Descent(12/99): loss=2.0673461964
Gradient Descent(13/99): loss=1.3476098843
Gradient Descent(14/99): loss=2.1680015315
Gradient Descent(15/99): loss=2.7306333926
Gradient Descent(16/99): loss=1.9052945848
Gradient Descent(17/99): loss=1.9181537753
Gradient Descent(18/99): loss=1.4767661456
Gradient Descent(19/99): loss=2.1310986616
Gradient Descent(20/99): loss=3.8156786178
Gradient Descent(21/99): loss=2.2539738750
Gradient Descent(22/99): loss=1.7614698252
Gradient Descent(23/9

In [12]:
w_ls, loss_ls = least_squares(y, tx)

In [13]:
w_rr, loss_rr = ridge_regression(y, tx, lambda_)

In [17]:
w_lr, loss_lr = logistic_regression(y, tx, initial_w, max_iters, gamma)

Gradient Descent(0/99): loss=173286.7951399862
Gradient Descent(1/99): loss=167371.2566432319
Gradient Descent(2/99): loss=163280.4137884185
Gradient Descent(3/99): loss=160612.3812308610
Gradient Descent(4/99): loss=160771.2364011554
Gradient Descent(5/99): loss=160476.0834131945
Gradient Descent(6/99): loss=158730.1683917697
Gradient Descent(7/99): loss=157753.1319766692
Gradient Descent(8/99): loss=160393.5211294697
Gradient Descent(9/99): loss=160487.2845154690
Gradient Descent(10/99): loss=160678.0608520149
Gradient Descent(11/99): loss=164032.4072089480
Gradient Descent(12/99): loss=160879.1782821260
Gradient Descent(13/99): loss=164339.1654566197
Gradient Descent(14/99): loss=161443.4559832202
Gradient Descent(15/99): loss=159483.7740409501
Gradient Descent(16/99): loss=159780.6176535089
Gradient Descent(17/99): loss=158461.4826320697
Gradient Descent(18/99): loss=157717.8254211573
Gradient Descent(19/99): loss=157836.4541002009
Gradient Descent(20/99): loss=157504.0120086611
Gr

In [18]:
w_rlr, loss_rlr = reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma)

Gradient Descent(0/99): loss=173286.7951399862
Gradient Descent(1/99): loss=178776.9669063461
Gradient Descent(2/99): loss=172877.0018529257
Gradient Descent(3/99): loss=168326.3077416788
Gradient Descent(4/99): loss=169184.2314204616
Gradient Descent(5/99): loss=164687.7576053823
Gradient Descent(6/99): loss=161443.2054102583
Gradient Descent(7/99): loss=159547.5273408344
Gradient Descent(8/99): loss=159705.6477917326
Gradient Descent(9/99): loss=162416.3108747509
Gradient Descent(10/99): loss=162753.2931191724
Gradient Descent(11/99): loss=160756.2211175978
Gradient Descent(12/99): loss=158906.9696986105
Gradient Descent(13/99): loss=157825.4655308444
Gradient Descent(14/99): loss=157719.7303032613
Gradient Descent(15/99): loss=157535.1685713945
Gradient Descent(16/99): loss=157594.0300478773
Gradient Descent(17/99): loss=157700.6334137765
Gradient Descent(18/99): loss=157854.9037800505
Gradient Descent(19/99): loss=157638.8563327049
Gradient Descent(20/99): loss=157773.8726524371
Gr

In [16]:
train_X = tx[:200000]
train_y = y[:200000]

test_X = tx[200000:]
test_y = y[200000:]

#w_lr, loss_lr = logistic_regression(train_y, train_X, initial_w, max_iters, gamma)

#preds = 1 / (1 + np.exp(-(test_X @ w_lr)))
#preds[preds <= 0.5] = -1
#preds[preds > 0.5] = 1

w_lr, loss_rr = ridge_regression(train_y, train_X, lambda_)

preds = test_X @ w_lr
preds[preds <= 0] = -1
preds[preds > 0] = 1

print(np.mean((preds == test_y)))

0.69238
